<a href="https://colab.research.google.com/github/EWOLO-JM/Master-M2-CIS/blob/main/CodeMethodeCISLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dissimulation du message**

In [ ]:
import cv2
import numpy as np
from google.colab import files
import io

def cover_image_preparation(cover_picture, m, n):
    cover_segments = []
    for i in range(0, cover_picture.shape[0], m):
        for j in range(0, cover_picture.shape[1], n):
            segment = cover_picture[i:i+m, j:j+n]
            lowest_value = np.min(segment)
            highest_value = np.max(segment)
            cover_segments.append((lowest_value, highest_value))
    return cover_segments


def normalize_image(image, target_size=(512, 512)):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

def secret_data_preparation(secret_data, num_segments):
    binary_secret_data = ''.join(format(ord(char), '07b') for char in secret_data)
    # Ajouter des zéros à la fin pour atteindre la longueur requise
    while len(binary_secret_data) < num_segments * 2:  # Multiplié par 2 pour prendre en compte les deux bits par segment
        binary_secret_data += '0'
    return binary_secret_data



def mapping(cover_segments, binary_secret_data, mapping_key):
    mapping_flags = []
    for i, (lowest, highest) in enumerate(cover_segments):
        mapped_bit = int(not (int(binary_secret_data[i]) ^ ((highest & 128) >> 7)))
        mapping_flags.append(mapped_bit)
    return mapping_flags

def embed_message(cover_image, secret_data, m, n, mapping_key):
    cover_segments = cover_image_preparation(cover_image, m, n)

    # Ajouter le nombre total de segments comme argument pour la préparation des données secrètes
    binary_secret_data = secret_data_preparation(secret_data, len(cover_segments))

    mapping_flags = mapping(cover_segments, binary_secret_data, mapping_key)

    stego_image = np.copy(cover_image)

    for i, (lowest, highest) in enumerate(cover_segments):
        stego_image[i // (cover_image.shape[1] // n), i % (cover_image.shape[1] // n)] = (stego_image[i // (cover_image.shape[1] // n), i % (cover_image.shape[1] // n)] & ~1) | mapping_flags[i]

    return stego_image

# Charger l'image de couverture depuis l'ordinateur local
uploaded = files.upload()
cover_image_path = list(uploaded.keys())[0]
cover_image_data = uploaded[cover_image_path]
cover_image = cv2.imdecode(np.frombuffer(cover_image_data, np.uint8), cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image a réussi
if cover_image is None:
    print("Erreur: Impossible de lire l'image de couverture.")
else:
    # Données secrètes à dissimuler
    secret_data = "Hello, Mr EWOLO !"

    # Paramètres pour la méthode CIHLHF
    m = 4
    n = 4
    mapping_key = [7, 8, 1, 2, 3, 4, 6, 5]

    # Dissimuler le message dans l'image de couverture
    stego_image = embed_message(cover_image, secret_data, m, n, mapping_key)

    # Enregistrer l'image stéganographique
    cv2.imwrite("/content/stego_image1.jpg", stego_image)


In [ ]:
import cv2
import numpy as np
from google.colab import files
import io

def normalize_image(image, target_size=(512, 512)):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

def cover_image_preparation(cover_picture, m, n):
    cover_segments = []
    for i in range(0, cover_picture.shape[0], m):
        for j in range(0, cover_picture.shape[1], n):
            segment = cover_picture[i:i+m, j:j+n]
            lowest_value = np.min(segment)
            highest_value = np.max(segment)
            cover_segments.append((lowest_value, highest_value))
    return cover_segments

def secret_data_preparation(secret_data, num_segments):
    binary_secret_data = ''.join(format(ord(char), '07b') for char in secret_data)
    # Ajouter des zéros à la fin pour atteindre la longueur requise
    while len(binary_secret_data) < num_segments * 2:  # Multiplié par 2 pour prendre en compte les deux bits par segment
        binary_secret_data += '0'
    return binary_secret_data

def mapping(cover_segments, binary_secret_data, mapping_key):
    mapping_flags = []
    for i, (lowest, highest) in enumerate(cover_segments):
        mapped_bit = int(not (int(binary_secret_data[i]) ^ ((highest & 128) >> 7)))
        mapping_flags.append(mapped_bit)
    return mapping_flags

def embed_message(cover_image, secret_data, m, n, mapping_key):
    # Normaliser l'image à la taille spécifiée (512x512)
    normalized_cover_image = normalize_image(cover_image)
    cover_segments = cover_image_preparation(normalized_cover_image, m, n)

    # Ajouter le nombre total de segments comme argument pour la préparation des données secrètes
    binary_secret_data = secret_data_preparation(secret_data, len(cover_segments))

    mapping_flags = mapping(cover_segments, binary_secret_data, mapping_key)

    stego_image = np.copy(normalized_cover_image)

    for i, (lowest, highest) in enumerate(cover_segments):
        row = i // (normalized_cover_image.shape[1] // n)
        col = i % (normalized_cover_image.shape[1] // n)
        stego_image[row * m:(row + 1) * m, col * n:(col + 1) * n] = (stego_image[row * m:(row + 1) * m, col * n:(col + 1) * n] & ~1) | mapping_flags[i]

    return stego_image

# Charger l'image de couverture depuis l'ordinateur local
uploaded = files.upload()
cover_image_path = list(uploaded.keys())[0]
cover_image_data = uploaded[cover_image_path]
cover_image = cv2.imdecode(np.frombuffer(cover_image_data, np.uint8), cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image a réussi
if cover_image is None:
    print("Erreur: Impossible de lire l'image de couverture.")
else:
    # Données secrètes à dissimuler
    secret_data = "Hello, Mr EWOLO !"

    # Paramètres pour la méthode CIHLHF
    m = 4
    n = 4
    mapping_key = [7, 8, 1, 2, 3, 4, 6, 5]

    # Dissimuler le message dans l'image de couverture
    stego_image = embed_message(cover_image, secret_data, m, n, mapping_key)

    # Enregistrer l'image stéganographique
    cv2.imwrite("/content/stego_image_normalized.jpg", stego_image)


Afficher l'image Stégo

In [ ]:
import cv2
from matplotlib import pyplot as plt

# Charger l'image stéganographique
stego_image_path = "/content/stego_image_normalized.jpg"
stego_image = cv2.imread(stego_image_path, cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image a réussi
if stego_image is None:
    print("Erreur: Impossible de lire l'image stéganographique.")
else:
    # Afficher l'image stéganographique
    plt.imshow(stego_image, cmap='gray')
    plt.title('Image stéganographique')
    plt.axis('off')
    plt.show()


Afficher les dimmenssions de l'image stégo

In [ ]:
print("Dimensions de l'image stéganographique:", stego_image.shape)

**Extracction du message**

In [ ]:
# Code d'extraction du message secret

In [ ]:
import cv2
import numpy as np
from google.colab import files

def normalize_image(image, target_size=(512, 512)):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

def cover_image_preparation(cover_picture, m, n):
    cover_segments = []
    for i in range(0, cover_picture.shape[0], m):
        for j in range(0, cover_picture.shape[1], n):
            segment = cover_picture[i:i+m, j:j+n]
            lowest_value = np.min(segment)
            highest_value = np.max(segment)
            cover_segments.append((lowest_value, highest_value))
    return cover_segments

def extract_message(stego_image, m, n, mapping_key):
    stego_segments = cover_image_preparation(stego_image, m, n)

    # Vérifier si les dimensions correspondent
    if len(stego_segments) != stego_image.size:
        print("Erreur: Les dimensions de l'image stéganographique ne correspondent pas.")
        return None

    extracted_binary_data = ""
    for i, (lowest, highest) in enumerate(stego_segments):
        extracted_bit = (stego_image[i // m, i % n] & 1) ^ ((highest & 128) >> 7)
        extracted_binary_data += str(extracted_bit)

    # Convertir les données binaires extraites en ASCII
    extracted_message = ""
    for i in range(0, len(extracted_binary_data), 7):
        ascii_code = int(extracted_binary_data[i:i+7], 2)
        extracted_message += chr(ascii_code)

    return extracted_message

# Charger l'image stéganographique depuis l'ordinateur local
uploaded = files.upload()
stego_image_path = list(uploaded.keys())[0]
stego_image_data = uploaded[stego_image_path]
stego_image = cv2.imdecode(np.frombuffer(stego_image_data, np.uint8), cv2.IMREAD_GRAYSCALE)

# Vérifier si la lecture de l'image stéganographique a réussi
if stego_image is None:
    print("Erreur: Impossible de lire l'image stéganographique.")
else:
    # Paramètres pour la méthode CIHLHF
    m = 4
    n = 4
    mapping_key = [7, 8, 1, 2, 3, 4, 6, 5]

    # Extraire le message secret de l'image stéganographique
    extracted_message = extract_message(stego_image, m, n, mapping_key)

    if extracted_message is not None:
        # Afficher le message extrait
        print("Message secret extrait:", extracted_message)
    else:
        print("Extraction du message échouée.")


Importer un fichier

In [ ]:
from google.colab import files
file = files.upload()